# Finetune a CNN on the Resisc45 dataset

This notebook uses some common CNN model provided by torchvision to finetune the model pretrained on ImageNet using the satellite images in the Resisc45 dataset. 

Most of the code used in this notebook has been taken from a tutorial in the official PyTorch documentation at https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html.

In [ ]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import models, transforms
import time
import os
import copy

from PIL import Image
from utils.utils import load_imagepaths_and_labels
from utils.utils import get_label_number
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

## 1. Define global variables and functions

Make sure to change these global variables to use the right CNN model, batch size, ...

In [ ]:
# Top level data directory.
data_dir = "./NWPU-RESISC45"
# train, val and test txt files lists
image_path_lists = {
    "train": "train_imagepaths.txt",
    "val": "val_imagepaths.txt",
    "test": "test_imagepaths.txt",
}
label_map_filename = "label_map.json"

# Models to choose from [squeezenet, shufflenet, mobilenet, mnasnet, resnet18, resnet50]
model_name = "resnet50"

# Number of classes in the dataset
num_classes = 45

# Batch size for training (change depending on how much memory you have)
batch_size = 8

# Number of epochs to train for
num_epochs = 15

# Flag for feature extracting. When False, we finetune the whole model,
# when True we only update the reshaped layer params
feature_extract = False

# Image size
input_size = 224

## 2. Define helpers functions and classes

In [ ]:
class Resisc45DataLoader(torch.utils.data.Dataset):
    """
    This is the main class that reads the Resisc45 dataset and return the
    images and labels, with the possibility of applying data augmentation.
    """

    def __init__(
        self,
        dataset_path,
        imagepaths,
        label_map_path,
        transform,
    ):
        self.dataset_path = dataset_path

        self.imagepaths_and_labels = load_imagepaths_and_labels(
            dataset_path, imagepaths,
        )
        self.label_map_path = label_map_path
        self.transform = transform

    def __getitem__(self, index):
        img_path = self.imagepaths_and_labels[index][0]
        text_label = self.imagepaths_and_labels[index][1]
        img = Image.open(os.path.join(self.dataset_path, img_path))

        label = get_label_number(
            self.dataset_path, self.label_map_path, text_label
        )

        return self.transform(img), label

    def __len__(self):
        return len(self.imagepaths_and_labels)

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

The following helper function sets the .requires_grad attribute of the parameters in the model to False when we are feature extracting. By default, when we load a pretrained model all of the parameters have .requires_grad=True, which is fine if we are training from scratch or finetuning. However, if we are feature extracting and only want to compute gradients for the newly initialized layer then we want all of the other parameters to not require gradients. 

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

## 3. Instantiate model and data loaders

In [ ]:
# Instantiate model and change last classification layer
if model_name == "squeezenet":
    model_ft = models.squeezenet1_0(pretrained=True)
    model_ft.classifier[1] = nn.Conv2d(
        512, num_classes, kernel_size=(1, 1), stride=(1, 1)
    )
elif model_name == "shufflenet":
    model_ft = models.shufflenet_v2_x1_0(pretrained=True)
    model_ft.fc = nn.Linear(1024, num_classes)
elif model_name == "mobilenet":
    model_ft = models.mobilenet_v2(pretrained=True)
    model_ft.classifier[1] = nn.Linear(1280, num_classes)
elif model_name == "mnasnet":
    model_ft = models.mnasnet1_0(pretrained=True)
    model_ft.classifier[1] = nn.Linear(1280, num_classes)
elif model_name == "resnet18":
    model_ft = models.resnet18(pretrained=True)
    model_ft.fc = nn.Linear(512, num_classes)
elif model_name == "resnet50":
    model_ft = models.resnet50(pretrained=True)
    model_ft.fc = nn.Linear(2048, num_classes)

# Freeze parameters
set_parameter_requires_grad(model_ft, feature_extract)

# Print the model we just instantiated
print(model_ft)
print(sum(p.numel() for p in model_ft.parameters() if p.requires_grad))

In [ ]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {
    x: Resisc45DataLoader(
        data_dir, 
        image_path_lists[x],
        label_map_filename,
        data_transforms[x]
    ) for x in ['train', 'val']
}
# Create training and validation dataloaders
dataloaders_dict = {
    x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=2) \
    for x in ['train', 'val']
}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## 4. Train model

In [ ]:
# Send the model to GPU
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

In [ ]:
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model_ft, hist = train_model(
    model_ft,
    dataloaders_dict,
    criterion,
    optimizer_ft,
    num_epochs=num_epochs,
    is_inception=(model_name=="inception")
)

## 5. Save Model

In [ ]:
torch.save(model_ft.state_dict(), model_name + "_resisc45_pyt_finetuned_weights.pth")